In [4]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.driver.host", "localhost").getOrCreate()
file_path = 'file:///C:/Users/seanz/VSCode_WS/BigData/NFL_pbp_2009-2019.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)



df.show()

+-------+----------+---------+---------+-------+------------+-------+-------------+------------+----------+-------------------------+----------------------+----------------------+---------+-----------+-----+---+---+----+----------+-------------------+------+-------+------+--------------------+---------+------------+-------+---------+-----------+--------+--------+-----------+-----------+-------------+---------+-----------------+------------+-------+-----------------+-------------+------------------+---------------------+-----------------------+-----------------------+-------+------------+-------+--------------------------+--------------------------+----------------+----------------+-------------+-------------+------------------+------------------+------------------+-----------------------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+----------------+-------------------------+-------------------+---

In [5]:
from pyspark.sql.functions import col

df.printSchema()

int_cols = [col_name for col_name, dtype in df.dtypes if dtype in ['int', 'bigint', 'double']]
df = df.fillna(0, subset=int_cols)

string_cols = [col_name for col_name, dtype in df.dtypes if dtype == 'string']
df = df.fillna("None", subset=string_cols)

missing_values_summary = df.select([col(c).isNull().cast("int").alias(c) for c in df.columns]).groupBy().sum()
missing_values_summary.show()

df = df.dropna(subset=['time'])

df.describe().show()

root
 |-- play_id: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- posteam: string (nullable = true)
 |-- posteam_type: string (nullable = true)
 |-- defteam: string (nullable = true)
 |-- side_of_field: string (nullable = true)
 |-- yardline_100: double (nullable = true)
 |-- game_date: date (nullable = true)
 |-- quarter_seconds_remaining: double (nullable = true)
 |-- half_seconds_remaining: double (nullable = true)
 |-- game_seconds_remaining: double (nullable = true)
 |-- game_half: string (nullable = true)
 |-- quarter_end: integer (nullable = true)
 |-- drive: integer (nullable = true)
 |-- sp: integer (nullable = true)
 |-- qtr: integer (nullable = true)
 |-- down: double (nullable = true)
 |-- goal_to_go: double (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- yrdln: string (nullable = true)
 |-- ydstogo: integer (nullable = true)
 |-- ydsnet: integer (nullable

In [6]:
columns = df.columns
print("Columns in the DataFrame:", columns)

Columns in the DataFrame: ['play_id', 'game_id', 'home_team', 'away_team', 'posteam', 'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'game_date', 'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet', 'desc', 'play_type', 'yards_gained', 'shotgun', 'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 'run_location', 'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result', 'two_point_conv_result', 'home_timeouts_remaining', 'away_timeouts_remaining', 'timeout', 'timeout_team', 'td_team', 'posteam_timeouts_remaining', 'defteam_timeouts_remaining', 'total_home_score', 'total_away_score', 'posteam_score', 'defteam_score', 'score_differential', 'posteam_score_post', 'defteam_score_post', 'score_differential_post', 'no_score_prob', 'opp_fg_prob', 'opp_s

In [7]:
colsToRemove = ['no_score_prob', 'opp_fg_prob', 'opp_safety_prob', 'opp_td_prob', 'fg_prob', 'safety_prob', 
'td_prob', 'extra_point_prob', 'two_point_conversion_prob', 'ep', 'epa', 'total_home_epa', 'total_away_epa', 
'total_home_rush_epa', 'total_away_rush_epa', 'total_home_pass_epa', 'total_away_pass_epa', 'air_epa', 'yac_epa', 
'comp_air_epa', 'comp_yac_epa', 'total_home_comp_air_epa', 'total_away_comp_air_epa', 'total_home_comp_yac_epa', 
'total_away_comp_yac_epa', 'total_home_raw_air_epa', 'total_away_raw_air_epa', 'total_home_raw_yac_epa', 
'total_away_raw_yac_epa', 'wp', 'def_wp', 'home_wp', 'away_wp', 'wpa', 'home_wp_post', 'away_wp_post', 
'total_home_rush_wpa', 'total_away_rush_wpa', 'total_home_pass_wpa', 'total_away_pass_wpa', 'air_wpa', 
'yac_wpa', 'comp_air_wpa', 'comp_yac_wpa', 'total_home_comp_air_wpa', 'total_away_comp_air_wpa', 
'total_home_comp_yac_wpa', 'total_away_comp_yac_wpa', 'total_home_raw_air_wpa', 'total_away_raw_air_wpa', 
'total_home_raw_yac_wpa', 'total_away_raw_yac_wpa', 'passer_player_id', 'passer_player_name', 'receiver_player_id', 
'receiver_player_name', 'rusher_player_id', 'rusher_player_name', 'lateral_receiver_player_id', 'lateral_receiver_player_name', 
'lateral_rusher_player_id', 'lateral_rusher_player_name', 'lateral_sack_player_id', 'lateral_sack_player_name', 
'interception_player_id', 'interception_player_name', 'lateral_interception_player_id', 'lateral_interception_player_name', 
'punt_returner_player_id', 'punt_returner_player_name', 'lateral_punt_returner_player_id', 'lateral_punt_returner_player_name', 
'kickoff_returner_player_name', 'kickoff_returner_player_id', 'lateral_kickoff_returner_player_id', 
'lateral_kickoff_returner_player_name', 'punter_player_id', 'punter_player_name', 'kicker_player_name', 
'kicker_player_id', 'own_kickoff_recovery_player_id', 'own_kickoff_recovery_player_name', 'blocked_player_id', 
'blocked_player_name', 'tackle_for_loss_1_player_id', 'tackle_for_loss_1_player_name', 'tackle_for_loss_2_player_id', 
'tackle_for_loss_2_player_name', 'qb_hit_1_player_id', 'qb_hit_1_player_name', 'qb_hit_2_player_id', 'qb_hit_2_player_name',
'forced_fumble_player_1_team', 'forced_fumble_player_1_player_id', 'forced_fumble_player_1_player_name', 
'forced_fumble_player_2_team', 'forced_fumble_player_2_player_id', 'forced_fumble_player_2_player_name', 
'solo_tackle_1_team', 'solo_tackle_2_team', 'solo_tackle_1_player_id', 'solo_tackle_2_player_id', 'solo_tackle_1_player_name',
'solo_tackle_2_player_name', 'assist_tackle_1_player_id', 'assist_tackle_1_player_name', 'assist_tackle_1_team', 
'assist_tackle_2_player_id', 'assist_tackle_2_player_name', 'assist_tackle_2_team', 'assist_tackle_3_player_id', 
'assist_tackle_3_player_name', 'assist_tackle_3_team', 'assist_tackle_4_player_id', 'assist_tackle_4_player_name', 
'assist_tackle_4_team', 'pass_defense_1_player_id', 'pass_defense_1_player_name', 'pass_defense_2_player_id', 
'pass_defense_2_player_name', 'fumbled_1_team', 'fumbled_1_player_id', 'fumbled_1_player_name', 'fumbled_2_player_id', 
'fumbled_2_player_name', 'fumbled_2_team', 'fumble_recovery_1_team', 'fumble_recovery_1_yards', 'fumble_recovery_1_player_id',
'fumble_recovery_1_player_name', 'fumble_recovery_2_team', 'fumble_recovery_2_yards', 'fumble_recovery_2_player_id', 
'fumble_recovery_2_player_name', 'penalty_player_id', 'penalty_player_name']

df = df[[col for col in df.columns if col not in colsToRemove]]
columns = df.columns
print("Columns in the DataFrame:", columns)

Columns in the DataFrame: ['play_id', 'game_id', 'home_team', 'away_team', 'posteam', 'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'game_date', 'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet', 'desc', 'play_type', 'yards_gained', 'shotgun', 'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 'run_location', 'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result', 'two_point_conv_result', 'home_timeouts_remaining', 'away_timeouts_remaining', 'timeout', 'timeout_team', 'td_team', 'posteam_timeouts_remaining', 'defteam_timeouts_remaining', 'total_home_score', 'total_away_score', 'posteam_score', 'defteam_score', 'score_differential', 'posteam_score_post', 'defteam_score_post', 'score_differential_post', 'punt_blocked', 'first_down_rush', 'fi

# Installed Packages:
## pip install pyspark, pip install hdfs

# Different Methods used from Thesis:
## Spark Session
## PySpark SQL to find missing values, Filled in values with 0 or none depending on data type instead of removing columns with mostly missing values (select, alias, groupby, sum)
## Removed rows with missing time values, these are the rows where there was no real play, usually end of quarter indicators
## Removed all columns that were data added in from different models (WP, EPA, WPA, Probability of events) and all columns that held player names. None of these will be needed for looking at play trends and future work

# Next Weeks Plan:
## Visualize/Summarize Play Type Data (Play Type Distribution in general, Play type vs quarter, play type vs different scores, Play type vs Down, Play type in 2-minutes remaining in half vs rest, play type vs offensive formation, play type vs field position, play type for Home and Away, Play Type distribution for different years). Possibly combine a few of these conditions to get a better idea (2 minute offense for winning team vs 2 minute offense for losing team should have vastly different play choices)
## Do i need to do correlation analysis?

In [ ]:
df_pandas = df.toPandas()

df_pandas.to_csv('NFL_PBP_V1.csv', index=False)